
Before using this tutorial, ensure that the following are on your system 

1. SteganoGAN IS INSTALLED
2. DATA FOR TRAINING IS LOADED


In the next step, we will import all the necessary components from SteganoGAN. Let us go through each of these components.


1. from steganogan import SteganoGAN

This imports the SteganoGAN class which has wrapper functions that allow you to use SteganoGAN. This wrapper class includes methods to create a SteganoGAN architecture, train that architecture, save the trained model, and then load and use trained models.


2. from steganogan.loader import DataLoader

The dataloader is used to load images from a selected database with different specified hyper parameters. You can tune these parameters when loading the images from the database. 

3. from steganogan.encoders import BasicEncoder, DenseEncoder

The encoders are the parts that 

4. from steganogan.decoders 

5. from steganogan.critics import BasicCritic

In [1]:
from steganogan import SteganoGAN
from steganogan.loader import DataLoader
from steganogan.encoders import BasicEncoder, DenseEncoder
from steganogan.decoders import BasicDecoder, DenseDecoder
from steganogan.critics import BasicCritic


In [ ]:
#####################################################################

DISCUSSS DATA LOADING 

#####################################################################

In [2]:
# Load the data
train = DataLoader('data/div2k/train/')
validation = DataLoader('data/div2k/val/')

In [ ]:
#####################################################################

DISCUSS EACH PARAMATER

#####################################################################

In [3]:
# Create the SteganoGAN instance
steganogan = SteganoGAN(1, BasicEncoder, BasicDecoder, BasicCritic, hidden_size=32, cuda=True, verbose=True)

Using CUDA device


In [ ]:
#####################################################################

DISCUSS EPOCH TRAINING 

#####################################################################

In [2]:
# Fit on the given data
steganogan.fit(train, validation, epochs=10)

NameError: name 'steganogan' is not defined

In [ ]:
#####################################################################

DISCUSS MODEL SAVING AND LOADING

#####################################################################

In [5]:
# Save the fitted model
steganogan.save('models/basic.steg')

In [6]:
# Load the model
steganogan = SteganoGAN.load('models/basic.steg')

Using CUDA device


In [7]:
# Encode a message in input.png
steganogan.encode('input.png', 'output.png', 'This is a super secret message!')

Encoding completed.


In [8]:
# Decode the message from output.png
steganogan.decode('output.png')

'This is a super secret message!'